In [1]:
# 데이터 만들기
from patsy import demo_data
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from io import StringIO

csv_data = StringIO("""
x1,x2,x3,x4,x5
1,0.1,"1",2019-01-01,A
2,,,2019-01-02,B
3,,"3",2019-01-03,C
,0.4,"4",2019-01-04,A
5,0.5,"5",2019-01-05,B
,,,2019-01-06,C
7,0.7,"7",,A
8,0.8,"8",2019-01-08,B
9,0.9,,2019-01-09,C
""")

df = pd.DataFrame(demo_data("x1", "x2", "x3", "x4", "x5"))
df

x1        x2        x3        x4        x5
0  1.764052 -0.977278  0.144044  0.333674 -2.552990
1  0.400157  0.950088  1.454274  1.494079  0.653619
2  0.978738 -0.151357  0.761038 -0.205158  0.864436
3  2.240893 -0.103219  0.121675  0.313068 -0.742165
4  1.867558  0.410599  0.443863 -0.854096  2.269755

데이터 분석을 시작할 때는 다양한 데이터를 하나의 데이터프레임에 넣고 시작하는 경우가 많다.  patsy 패키지가 제공하는 `dmatrx()` 함수를 사용하면 데이터 프레임에 상수항을 추가하거나 원하는 데이터만 선택하거나 변형할 수 있다. `dmatrx()` 함수에 모형 정의 문자열 `formula`와 원 데이터를 담은 데이터프레임 data을 입력하면 `formula`에서 지정한 대로 변환된 데이터 data_transformed를 출력한다.

```
data_transformed = dmatrix(formula, data)
```

formula는 데이터 열 이름 기반으로 구성된 문자열이다. 선택하고자 하는 데이터 열 이름을 `+`로 연결한 formula 문자열을 입려하면 자동으로 해당 데이터만 뽑아준다. 예를 들어 전체 데이터 중 x1만을 뽑고 싶으면 formula 문자열에 `x1 + 0`이라고 입력한다.

# 원하는 col만 선택하기

In [2]:
from patsy import dmatrix 

x = dmatrix("x1 -1 ", data=df)
x

DesignMatrix with shape (5, 1)
       x1
  1.76405
  0.40016
  0.97874
  2.24089
  1.86756
  Terms:
    'x1' (column 0)

전체 데이터 중 x1, x2, x3를 뽑고 싶으면 formula 문자열에 `x1 + x2 + x3 - 1`이라고 입력한다.

In [3]:
dmatrix("x1 + x2 + x3 - 1", data=df,return_type="dataframe")
# return_type="dataframe" 을 넣음으로서 matrix 가 아닌 df 으로 만들수도 있다.
# 하지만 desgnmatrix 를 만드는거라 matrix multplication 을 하는 경우가 생겨서 추천하지는 않는다.

x1        x2        x3
0  1.764052 -0.977278  0.144044
1  0.400157  0.950088  1.454274
2  0.978738 -0.151357  0.761038
3  2.240893 -0.103219  0.121675
4  1.867558  0.410599  0.443863

`- 1`은 1로 구성된 상수항을 넣지 말라는 뜻이다.(+0 도 같은뜻) 만약 이 부분이 없으면 patsy는 자동으로 가장 앞에 Intercept란 이름의 상수항 데이터를 추가한다. 

# intercept 항 추가하기

In [4]:
dmatrix("x1 + x2 + x3", data=df)

DesignMatrix with shape (5, 4)
  Intercept       x1        x2       x3
          1  1.76405  -0.97728  0.14404
          1  0.40016   0.95009  1.45427
          1  0.97874  -0.15136  0.76104
          1  2.24089  -0.10322  0.12168
          1  1.86756   0.41060  0.44386
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'x2' (column 2)
    'x3' (column 3)

`dmatrix()` 함수는 변수를 어떤 함수에 넣어서 다른 값으로 만드는 수학 변환(transform)도 가능하다. 

 # col을 수학연산으로 변환시키기

In [5]:
dmatrix("x1 + np.log(np.abs(x2))", df)

DesignMatrix with shape (5, 3)
  Intercept       x1  np.log(np.abs(x2))
          1  1.76405            -0.02298
          1  0.40016            -0.05120
          1  0.97874            -1.88811
          1  2.24089            -2.27090
          1  1.86756            -0.89014
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'np.log(np.abs(x2))' (column 2)

numpy 함수 뿐 아니라 사용자 정의 함수도 사용할 수 있다.

In [6]:
def ten_times(x):
    return 10 * x

dmatrix("ten_times(x1)", df)

DesignMatrix with shape (5, 2)
  Intercept  ten_times(x1)
          1       17.64052
          1        4.00157
          1        9.78738
          1       22.40893
          1       18.67558
  Terms:
    'Intercept' (column 0)
    'ten_times(x1)' (column 1)

# 기존의 col을 조합해서 새로운 데이터 만들기

기존의 데이터를 조합 연산하여 새로운 데이터를 만드는 것도 가능하다. 특히 두 변수를 곱해서 만들어지는 새로운 변수를 **상호작용(interaction)**이라고 한다. 상호작용은 `:` 기호를 사용하여 만들어진다. 

In [7]:
dmatrix("x1 + x2 + x1:x2 + 0", df)

DesignMatrix with shape (5, 3)
       x1        x2     x1:x2
  1.76405  -0.97728  -1.72397
  0.40016   0.95009   0.38018
  0.97874  -0.15136  -0.14814
  2.24089  -0.10322  -0.23130
  1.86756   0.41060   0.76682
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'x1:x2' (column 2)

두 변수와 상호작용을 한꺼번에 표시할 때는 `*`를 사용한다

In [8]:
dmatrix("x1 * x2 * x3 - 1 ", df)

DesignMatrix with shape (5, 7)
       x1        x2     x1:x2       x3    x1:x3     x2:x3  x1:x2:x3
  1.76405  -0.97728  -1.72397  0.14404  0.25410  -0.14077  -0.24833
  0.40016   0.95009   0.38018  1.45427  0.58194   1.38169   0.55289
  0.97874  -0.15136  -0.14814  0.76104  0.74486  -0.11519  -0.11274
  2.24089  -0.10322  -0.23130  0.12168  0.27266  -0.01256  -0.02814
  1.86756   0.41060   0.76682  0.44386  0.82894   0.18225   0.34036
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'x1:x2' (column 2)
    'x3' (column 3)
    'x1:x3' (column 4)
    'x2:x3' (column 5)
    'x1:x2:x3' (column 6)

상호작용을 제외한  경우에는 `I()`라는 연산자를 사용하여 연산과정을 명시해야 한다. 예를 들어 두 변수 x1과 x2를 더하여 새로운 데이터를 만들고 싶다면 다음과 같이 한다.

In [9]:
dmatrix("x1 + x2 + I(x1 + x2) + 0", df,return_type="dataframe")
# 대문자 i 

x1        x2  I(x1 + x2)
0  1.764052 -0.977278    0.786774
1  0.400157  0.950088    1.350246
2  0.978738 -0.151357    0.827381
3  2.240893 -0.103219    2.137674
4  1.867558  0.410599    2.278156

# 평균 0 으로 만드는 스케일링하기

선형회귀분석을 할 때는 조건수(condition number)의 영향때문에 데이터의 평균을 0으로 표준편차를 1로 만드는 **스케일링(scaling)** 작업을 하는 것이 분석 결과의 품질을 높일 수 있다. 

In [10]:
dmatrix("I(x1-np.mean(x1))-1",df)

DesignMatrix with shape (5, 1)
  I(x1 - np.mean(x1))
              0.31377
             -1.05012
             -0.47154
              0.79061
              0.41728
  Terms:
    'I(x1 - np.mean(x1))' (column 0)

# train set 의 scaling 설정 그대로 test set에 적용하기

In [11]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score

# 붓꽃 데이터 세트를 로딩합니다.
iris = load_iris()
iris_data = iris.data
iris_label = iris.target
# 데이터를 train set 과 test set 으로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(iris_data, iris_label,
                                                    test_size=0.2, random_state=2)

In [12]:
#MinMaxScaler() Scaler객체 생성.
scaler = MinMaxScaler()
# 학습 데이터에 대해서 fit(), transform() 수행.
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
# 그냥 fit()만 쓰면 정규화 즉, 통계에서 정규분포를 만들게 하기 위해서 𝑥 값에서 평균을 빼고 그 값을 다시 표준편차로 나누어주는 작업

In [13]:
# 테스트 데이터에서는 다시 fit(), transform()이나 fit_transform()을 수행하지 않고 transform만 수행.
scaled_X_test = scaler.transform(X_test)
# 그래야 train data 에서 쓴 scaling 을 그대로 쓴다.

In [14]:
lr = LogisticRegression()
lr.fit(scaled_X_train, y_train)
pred = lr.predict(scaled_X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9333


fit_transform()을 fit() 과 transform() 함께 수행하는 메소드 입니다. 기존의 fit() 과 transform() 각각을 수행하는 번거로움을 줄여줍니다.  위 코드를 예로 들면

scaler.fit(X_train)

scaled_X_train = scaler.transform(X_train)은

=> scaled_X_train = scaler.fit_transform(X_train) 과 같이 한 라인으로 대체 될 수 있습니다. 

하지만 테스트 데이터에 scaled_X_test = scaler.fit_transform(X_test)를 적용해서는 안됩니다. 이를 수행하면 scaler 객체가

기존에 학습 데이터에 fit 했던 기준을 모두 무시하고 다시 테스트 데이터를 기반으로 기준을 적용하기 때문입니다. 

때문에 테스트 데이터에 fit_transform()을 적용해서는 안됩니다.

이런 번거로움을 피하기 위해 학습과 테스트 데이터로 나누기 전에 먼저 Scaling등의 데이터 전처리를 해주는 것이 좋습니다. 아래 코드에

서는 학습과 테스트 데이터로 나누기 전에 Scaling을 적용합니다. fit(), transform()을 순차적으로 써도 좋고 fit_transform()으로 변환

해도 무방합니다.

# scaling 을 전체 데이터에 대해 먼저 해버리기

In [15]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score

# 붓꽃 데이터 세트를 로딩합니다.
iris = load_iris()
iris_data = iris.data
iris_label = iris.target

# MinMaxScaler() Scaler객체 생성.
scaler = MinMaxScaler()
# 학습 데이터, 테스트 데이터로 나누기 전에 fit과 transform 으로 변환. fit_transform() 도 무방. 
scaler.fit(iris.data)
scaled_iris_data = scaler.transform(iris.data)
#이렇게 하면 먼저 전체 데이터에 대해 scaling 하게 된다.

# scale된 iris_data로 학습과 테스트 데이터를 나눔.
X_train, X_test, y_train, y_test = train_test_split(scaled_iris_data, iris_label,
                                                    test_size=0.2, random_state=2)

lr = LogisticRegression()
lr.fit(scaled_X_train, y_train)
pred = lr.predict(scaled_X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9333


# standardize(): 평균을 0으로하고 표준편차를 1로 스케일링

In [16]:
X = np.arange(-3,5).reshape(-1, 1)  
X

array([[-3],
       [-2],
       [-1],
       [ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4]])

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit_transform(X)

array([[-1.52752523],
       [-1.09108945],
       [-0.65465367],
       [-0.21821789],
       [ 0.21821789],
       [ 0.65465367],
       [ 1.09108945],
       [ 1.52752523]])

평균을 제거하고 데이터를 단위 분산으로 조정한다. 그러나 이상치가 있다면 평균과 표준편차에 영향을 미쳐 변환된 데이터의 구조는 매우 달라지게 된다.
따라서 이상치가 있는 경우 균형 잡힌 척도를 보장할 수 없다.

# MinMaxScaler() : 값이 0~1사이에 있도록 스케일링


모든 feature 값이 0~1사이에 있도록 데이터를 재조정한다. 다만 이상치가 있는 경우 변환된 값이 매우 좁은 범위로 압축될 수 있다.

아웃라이어의 존재에 매우 민감하다.

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit_transform(X)

array([[0.        ],
       [0.14285714],
       [0.28571429],
       [0.42857143],
       [0.57142857],
       [0.71428571],
       [0.85714286],
       [1.        ]])

# MaxAbsScaler() : 값이 -1 ~ 1 사이에 있도록 스케일링 

다만 이상치가 있는 경우 변환된 값이 매우 좁은 범위로 압축될 수 있다.

즉, MinMaxScaler 역시 아웃라이어의 존재에 매우 민감하다.

In [19]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
scaler.fit_transform(X)

array([[-0.75],
       [-0.5 ],
       [-0.25],
       [ 0.  ],
       [ 0.25],
       [ 0.5 ],
       [ 0.75],
       [ 1.  ]])

# RobustScaler : 이상치의 영향을 최소화

이 때는 `RobustScaler` 클래스를 사용한다. 이 클래스는 중앙값이 0, IQR(interquartile range)이 1이 되도록 변환하기 때문에 아웃라이어가 섞여 있어도 대부분의 데이터는 0 주위에 남아있게 된다.

IQR = Q3 - Q1 : 즉, 25퍼센타일과 75퍼센타일의 값

In [20]:
X2 = np.vstack([X,[[10],[14]]])

In [21]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit_transform(X2)

array([[-1.        ],
       [-0.77777778],
       [-0.55555556],
       [-0.33333333],
       [-0.11111111],
       [ 0.11111111],
       [ 0.33333333],
       [ 0.55555556],
       [ 1.88888889],
       [ 2.77777778]])

# 원래의 scale 로 변환하기

In [23]:
X = np.arange(-3,5).reshape(-1, 1)  
X

array([[-3],
       [-2],
       [-1],
       [ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4]])

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)

In [28]:
# 위와같이 scaled 된 x 를 다시 inverse 로 돌려버리면 된다.
scaler.inverse_transform(scaled_X)

array([[-3.],
       [-2.],
       [-1.],
       [ 0.],
       [ 1.],
       [ 2.],
       [ 3.],
       [ 4.]])

# Binarizer : binary value 를 가지게 변환

In [28]:
from sklearn.preprocessing import Binarizer
scaler = Binarizer(threshold=2) # 2를 넘으면 1/미만이면0
scaler.fit_transform(X)

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1]])

# LabelEncoder : categorical data를 value 로 변환

In [88]:
titanic = sns.load_dataset("titanic")
titanic.head()

survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True

In [89]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
titanic['class']=Encoder.fit_transform(titanic['class'])
titanic.head()

survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S      2   
1         1       1  female  38.0      1      0  71.2833        C      0   
2         1       3  female  26.0      0      0   7.9250        S      2   
3         1       1  female  35.0      1      0  53.1000        S      0   
4         0       3    male  35.0      0      0   8.0500        S      2   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True

# OneHotEncoder : value 를 onehotencoder

이 sklearn 의 onehotencoder 의 경우 수치형만 encding 이 가능해서, 먼저 label encoder를 해야한다.

In [117]:
from sklearn.preprocessing import OneHotEncoder
Encoder = OneHotEncoder() 
onehot = Encoder.fit_transform(titanic[['class']]).toarray() # toarray 로 해야 잘 array 로 변환된다.
pd.DataFrame(onehot,columns=['third','first','second']).head() 

C:\Users\Han\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


third  first  second
0    0.0    0.0     1.0
1    1.0    0.0     0.0
2    0.0    0.0     1.0
3    1.0    0.0     0.0
4    0.0    0.0     1.0

# OneHotEncode : pd.get_dummies 이용

In [121]:
pd.get_dummies(titanic['embark_town']) . head()

Cherbourg  Queenstown  Southampton
0          0           0            1
1          1           0            0
2          0           0            1
3          0           0            1
4          0           0            1

# 기존의 col 을 polynomial matrix 로 변환하기

preprocessing 서브패키지는 데이터 변환을 위한 `FunctionTransformer` 클래스와 `PolynomialFeatures` 클래스도 제공한다.

`PolynomialFeatures` 클래스는 입력 데이터 $x$를 다음과 같이 여러개의 다항식으로 변환한다.

$$ x \;\; \rightarrow \;\; [ 1, x, x^2, x^3, \cdots ] $$

다음과 같은 입력 인수를 가진다.

* `degree` : 차수
* `include_bias` : 상수항 생성 여부


In [229]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
poly.fit_transform(X)

array([[ 1., -3.,  9.],
       [ 1., -2.,  4.],
       [ 1., -1.,  1.],
       [ 1.,  0.,  0.],
       [ 1.,  1.,  1.],
       [ 1.,  2.,  4.],
       [ 1.,  3.,  9.],
       [ 1.,  4., 16.]])

# 기존의 col 을 사용자 지정함수 matrix 로 변환하기

`FunctionTransformer` 클래스는 사용자가 지정한 함수를 사용하여 입력값 $x$를 변환한다.

$$ x \;\; \rightarrow \;\; [ f_1(x),  f_2(x),  f_3(x),  \cdots ] $$

데이터 변환은 비선형 회귀분석에서 원하는 목표값을 더 잘 예측하기 위한 새로운 데이터를 만들 때 사용된다. 예를 들어 '360도'와 같은 각도 데이터는 그 자체로 예측문제의 입력값으로 넣을 수 없다. 0도와 360도, 10도와 370도는 사실 같은 각도지만 다른 숫자로 표현되기 때문이다. 이 때는 각도 $\theta$를 다음과 같이 삼각함수값의 쌍으로 바꾸면 같은 각도를 같은 숫자쌍으로 표현할 수 있다.

$$ \theta \;\; \rightarrow \;\; (\sin\theta, \cos\theta) $$

다음 예제 코드에서 원래 데이터 `X`는 각도 표시로 되어 있다. 

In [230]:
X = 90 * np.arange(9).reshape(-1, 1) 
X

array([[  0],
       [ 90],
       [180],
       [270],
       [360],
       [450],
       [540],
       [630],
       [720]])

하지만 `X2`는 삼각함수로 변환되었다. 각도 0도와 각도 360도가 모두 같은 값인 (0,1)로 표현된 것을 확인할 수 있다. 

In [231]:
from sklearn.preprocessing import FunctionTransformer

def degree2sincos(X):
    x0 = np.sin(X * np.pi / 180)
    x1 = np.cos(X * np.pi / 180)
    X_new = np.hstack([x0, x1])
    return X_new

X2 = FunctionTransformer(degree2sincos).fit_transform(X)
X2

C:\Users\Han\Anaconda3\lib\site-packages\sklearn\preprocessing\_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
C:\Users\Han\Anaconda3\lib\site-packages\sklearn\preprocessing\_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


array([[ 0.0000000e+00,  1.0000000e+00],
       [ 1.0000000e+00,  6.1232340e-17],
       [ 1.2246468e-16, -1.0000000e+00],
       [-1.0000000e+00, -1.8369702e-16],
       [-2.4492936e-16,  1.0000000e+00],
       [ 1.0000000e+00,  3.0616170e-16],
       [ 3.6739404e-16, -1.0000000e+00],
       [-1.0000000e+00, -4.2862638e-16],
       [-4.8985872e-16,  1.0000000e+00]])